<a href="https://colab.research.google.com/github/SnehaAgarwal07/challenges/blob/master/WGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import expand_dims
from numpy import mean
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras import backend
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from keras.constraints import Constraint
from matplotlib import pyplot

In [ ]:

params = {}
params['1_n_features'] = train_data.shape[1]
params['1.5_n_features'] = round(1.5 * train_data.shape[1])
params['2_n_features'] = 2* train_data.shape[1]


In [ ]:
# clip model weights to a given hypercube
class ClipConstraint(Constraint):
	# set clip value when initialized
	def __init__(self, clip_value):
		self.clip_value = clip_value
 
	# clip model weights to hypercube
	def __call__(self, weights):
		return backend.clip(weights, -self.clip_value, self.clip_value)
 
	# get the config
	def get_config(self):
		return {'clip_value': self.clip_value}
 
# calculate wasserstein loss
def wasserstein_loss(y_true, y_pred):
	return backend.mean(y_true * y_pred)

In [ ]:
# define the standalone critic model
def define_critic(in_shape):
  # weight initialization
  init = RandomNormal(stddev=0.02)
  # weight constraint
  const = ClipConstraint(0.01)
  # define model
  model = Sequential()

  model.add(Dense(64), kernel_initializer=init, kernel_constraint=const, input_shape=in_shape)
  model.add(LeakyReLU(alpha=0.2))

  model.add(Dense(128), kernel_initializer=init, kernel_constraint=const)
  model.add(LeakyReLU(alpha=0.2))

  model.add(Dense(256), kernel_initializer=init, kernel_constraint=const)
  model.add(LeakyReLU(alpha=0.2))
  # scoring, linear activation
  model.add(Flatten())
  model.add(Dense(1))
  # compile model
  opt = RMSprop(lr=0.00005)
  model.compile(loss=wasserstein_loss, optimizer=opt)
  return model

In [ ]:
def define_generator(latent_dim, params):
  
  # weight initialization
  init = RandomNormal(stddev=0.02)
  # define model
  model = Sequential()

  model.add(Dense(params['2_n_features']), kernel_initializer=init, input_dim=latent_dim, activation="relu")

  model.add(Dense(params['1.5_n_features']), kernel_initializer=init, activation="relu")

  model.add(Dense(params['1_n_features']), kernel_initializer=init, activation="sigmoid")
  model.add(sigmoid())
  return model

In [ ]:
# define the combined generator and critic model, for updating the generator
def define_gan(generator, critic):
	# make weights in the critic not trainable
	critic.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the critic
	model.add(critic)
	# compile model
	opt = RMSprop(lr=0.00005)
	model.compile(loss=wasserstein_loss, optimizer=opt)
	return model